## Goal: NSE comparison of NWM streamflow to gaged streamflow at one location
USGS Gage ID: 08076997 [Clear Ck at Mykawa St nr Pearland, TX](https://waterdata.usgs.gov/monitoring-location/08076997/?agency_cd=USGS#parameterCode=00065&period=P7D)(1979 to 2020)     
NWM Reach ID: 1562342 (2006 to Present)

*By Stephanie Brady and Emad Ahmed (TIAER @ Tarleton State University)*  
Credits: [James McCreight](https://nbviewer.org/github/NCAR/rechunk_retro_nwm_v21/blob/main/notebooks/usage_example_streamflow_timeseries.ipynb)

Running in Anaconda Navigator, using the environment "NWM_Zarr_v2.1.yaml" (available in the TIAER repository).

First, import the required packages.

In [1]:
from climata.usgs import DailyValueIO
from dask.distributed import Client
import fsspec
import holoviews as hv
import hvplot
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd 
import xarray as xr

ImportError: dask.distributed is not installed.

Please either conda or pip install distributed:

  conda install dask distributed             # either conda install
  python -m pip install "dask[distributed]" --upgrade    # or pip install

Setup Dask.distributed the [Easy Way](https://distributed.dask.org/en/stable/quickstart.html#setup-dask-distributed-the-easy-way)

In [ ]:
from dask.distributed import Client, progress
client = Client()
client

## 1) Download flow data from the [NWM model output data version 2.1 in Zarr format](https://registry.opendata.aws/nwm-archive/)


Point to the AWS CLI (Amazon Web Services Command Line Interface) where the data is storred in Zarr format.

In [ ]:
url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'

Print the [CPU Time and Wall Time](https://ipython.readthedocs.io/en/stable/interactive/magics.html?highlight=%25time#magic-time)   
Load and decode a dataset from a Zarr store [source](https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html)

In [ ]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)

Print the dataset

In [ ]:
ds

Identify what reach or reaches for which the NWM streamflows will be downloaded.     
For this example, we are using the USGS Gage `Clear Ck at Mykawa St nr Pearland, TX`; the associated Reach ID was found using the [NWM Map](https://water.noaa.gov/map).<br>   
For multiple reaches, use the syntax "reaches = np.sort(np.array([x1,x2,...]))".

In [ ]:
reach = np.sort(np.array([1562342]))

Print the wall time.   
Download the actual flows (most time-consuming step, takes about 9 mins on my computer).

In [ ]:
%%time
dsflows = ds.streamflow.sel(feature_id=reach).compute()

Convert the dataset array into a dataframe.

In [ ]:
streamflow_nwm_df= dsflows.to_pandas()

Print the dataframe header.Results are in cms.

In [ ]:
streamflow_nwm_df

Export the dataframe to a csv file. 

In [ ]:
streamflow_nwm_df.to_csv(r"C:\Users\sbrady\Downloads\NwmRetro2-1ZarrFlows_Mykawa.csv")

## 2) Download streamflow data from the USGS

In [ ]:
cfs_2_cms = 0.028316846592

hv.extension('bokeh')
hv.opts.defaults(
    hv.opts.Scatter(width=700, height=500) )
pd.options.plotting.backend = 'holoviews'


In [ ]:
usgs_station_id = "13317000"
param_id = "00060"  # streamflow in ft3/s
data = DailyValueIO(
    start_date="2016-01-01",
    end_date="2020-12-31",
    station=usgs_station_id,
    parameter=param_id,)

In [ ]:
# create lists of date-flow values
streamflow_usgs_d = {}
for series in data:
    streamflow_usgs_d['streamflow_obs'] = [r[1] * cfs_2_cms for r in series.data]
    streamflow_usgs_d['time'] = [pd.to_datetime(r[0]) for r in series.data]
    
streamflow_usgs_df = pd.DataFrame(streamflow_usgs_d).set_index('time')

In [ ]:
streamflow_usgs_df.plot.scatter(x='time', y='streamflow_obs')

Combine Plots by water year

In [ ]:
combo_df = (
    streamflow_nwm_df
    .join(streamflow_usgs_df, how='outer')
    .rename(columns={'streamflow': 'NWM v2.1', 'streamflow_obs': 'observed'}))

## 3)NSE

In [ ]:
import hydroeval as he

simulations = [5.3, 4.2, 5.7, 2.3]
evaluations = [4.7, 4.3, 5.5, 2.7]

nse = he.evaluator(he.nse, simulations, evaluations)
nse